# < QUBO model for 2x2 system of linear equations >

The following python code implements the QUBO formulation of system of linear equations using 2 variables.

The code also prints out the code for implementing the QUBO model on a D-Wave 2000Q quantum annealer.

Here, the QUBO formulation uses the following two relations on the set of utilized qubits:
* q^2 = q (binary qubits)
* q_i^+ * q_j^- = 0 (the product of positive digits and negative digits is equal to 0)

In [1]:
import numpy as np
import random, math
import copy

Dimension = 2
qubits = 2
A = np.array([[3, 1], [-1, 2]])
b = np.array([-1, 5])

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 4*i + l
            po2 = 4*i + l + 2
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 4*i + l1
                po2 = 4*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 4*i + l1 + 2
                po4 = 4*i + l2 + 2
                QM[po3][po4] = QM[po3][po4] + qcef
                
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 4*i + l1
                    po2 = 4*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 4*i + l1 + 2
                    po4 = 4*j + l2 + 2
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 4*i + l1
                    po6 = 4*j + l2 + 2
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 4*i + l1 + 2
                    po8 = 4*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)

# Print Python code for the run in D-Wave quantum processing unit
print("\n\nfrom dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)") 

# Matrix Q is
[[ 26.  40.   0.   0.   2.   4.  -2.  -4.]
 [  0.  72.   0.   0.   4.   8.  -4.  -8.]
 [  0.   0.  -6.  40.  -2.  -4.   2.   4.]
 [  0.   0.   0.   8.  -4.  -8.   4.   8.]
 [  0.   0.   0.   0. -13.  20.   0.   0.]
 [  0.   0.   0.   0.   0. -16.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  23.  20.]
 [  0.   0.   0.   0.   0.   0.   0.  56.]]


from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0  0  0  1  0  0  1  0  0  -26.0     837     0.0
1  0  0  1  0  1  0  0  0  -21.0      47     0.0
2  0  0  1  0  1  1  0  0  -21.0      77     0.0
3  0  0  0  1  0  1  0  0  -16.0      20     0.0
4  0  0  0  0  0  1  0  0  -16.0      14     0.0
6  0  0  0  0  0  1  0  0  -16.0       1   0.125
5  0  0  0  0  1  0  0  0  -13.0       3     0.0
7  0  0  1  0  0  1  1  0   -1.0       1     0.0
['BINARY', 8 rows, 1000 samples, 8 variables]


In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):8, ('q2','q2'):96/5, ('q3','q3'):256/5, ('q4','q4'):768/5, ('q5','q5'):512, ('q6','q6'):9216/5, ('q7','q7'):34816/5, ('q8','q8'):135168/5, ('q9','q9'):106496, ('q10','q10'):2113536/5, ('q11','q11'):-24/5, ('q12','q12'):-32/5, ('q13','q13'):0, ('q14','q14'):256/5, ('q15','q15'):1536/5, ('q16','q16'):7168/5, ('q17','q17'):6144, ('q18','q18'):126976/5, ('q19','q19'):516096/5, ('q20','q20'):2080768/5, ('q21','q21'):-882/125, ('q22','q22'):-1568/125, ('q23','q23'):-2352/125, ('q24','q24'):-1568/125, ('q25','q25'):9408/125, ('q26','q26'):68992/125, ('q27','q27'):338688/125, ('q28','q28'):1480192/125, ('q29','q29'):6171648/125, ('q30','q30'):25188352/125, ('q31','q31'):1078/125, ('q32','q32'):2352/125, ('q33','q33'):5488/125, ('q34','q34'):14112/125, ('q35','q35'):40768/125, ('q36','q36'):131712/125, ('q37','q37'):464128/125, ('q38','q38'):1731072/125, ('q39','q39'):6673408/125, ('q40','q40'):26191872/125}

quadratic = {('q1','q2'):32/5, ('q1','q3'):64/5, ('q1','q4'):128/5, ('q1','q5'):256/5, ('q1','q6'):512/5, ('q1','q7'):1024/5, ('q1','q8'):2048/5, ('q1','q9'):4096/5, ('q1','q10'):8192/5, ('q2','q3'):128/5, ('q2','q4'):256/5, ('q2','q5'):512/5, ('q2','q6'):1024/5, ('q2','q7'):2048/5, ('q2','q8'):4096/5, ('q2','q9'):8192/5, ('q2','q10'):16384/5, ('q3','q4'):512/5, ('q3','q5'):1024/5, ('q3','q6'):2048/5, ('q3','q7'):4096/5, ('q3','q8'):8192/5, ('q3','q9'):16384/5, ('q3','q10'):32768/5, ('q4','q5'):2048/5, ('q4','q6'):4096/5, ('q4','q7'):8192/5, ('q4','q8'):16384/5, ('q4','q9'):32768/5, ('q4','q10'):65536/5, ('q5','q6'):8192/5, ('q5','q7'):16384/5, ('q5','q8'):32768/5, ('q5','q9'):65536/5, ('q5','q10'):131072/5, ('q6','q7'):32768/5, ('q6','q8'):65536/5, ('q6','q9'):131072/5, ('q6','q10'):262144/5, ('q7','q8'):131072/5, ('q7','q9'):262144/5, ('q7','q10'):524288/5, ('q8','q9'):524288/5, ('q8','q10'):1048576/5, ('q9','q10'):2097152/5, ('q11','q12'):32/5, ('q11','q13'):64/5, ('q11','q14'):128/5, ('q11','q15'):256/5, ('q11','q16'):512/5, ('q11','q17'):1024/5, ('q11','q18'):2048/5, ('q11','q19'):4096/5, ('q11','q20'):8192/5, ('q12','q13'):128/5, ('q12','q14'):256/5, ('q12','q15'):512/5, ('q12','q16'):1024/5, ('q12','q17'):2048/5, ('q12','q18'):4096/5, ('q12','q19'):8192/5, ('q12','q20'):16384/5, ('q13','q14'):512/5, ('q13','q15'):1024/5, ('q13','q16'):2048/5, ('q13','q17'):4096/5, ('q13','q18'):8192/5, ('q13','q19'):16384/5, ('q13','q20'):32768/5, ('q14','q15'):2048/5, ('q14','q16'):4096/5, ('q14','q17'):8192/5, ('q14','q18'):16384/5, ('q14','q19'):32768/5, ('q14','q20'):65536/5, ('q15','q16'):8192/5, ('q15','q17'):16384/5, ('q15','q18'):32768/5, ('q15','q19'):65536/5, ('q15','q20'):131072/5, ('q16','q17'):32768/5, ('q16','q18'):65536/5, ('q16','q19'):131072/5, ('q16','q20'):262144/5, ('q17','q18'):131072/5, ('q17','q19'):262144/5, ('q17','q20'):524288/5, ('q18','q19'):524288/5, ('q18','q20'):1048576/5, ('q19','q20'):2097152/5, ('q21','q22'):392/125, ('q21','q23'):784/125, ('q21','q24'):1568/125, ('q21','q25'):3136/125, ('q21','q26'):6272/125, ('q21','q27'):12544/125, ('q21','q28'):25088/125, ('q21','q29'):50176/125, ('q21','q30'):100352/125, ('q22','q23'):1568/125, ('q22','q24'):3136/125, ('q22','q25'):6272/125, ('q22','q26'):12544/125, ('q22','q27'):25088/125, ('q22','q28'):50176/125, ('q22','q29'):100352/125, ('q22','q30'):200704/125, ('q23','q24'):6272/125, ('q23','q25'):12544/125, ('q23','q26'):25088/125, ('q23','q27'):50176/125, ('q23','q28'):100352/125, ('q23','q29'):200704/125, ('q23','q30'):401408/125, ('q24','q25'):25088/125, ('q24','q26'):50176/125, ('q24','q27'):100352/125, ('q24','q28'):200704/125, ('q24','q29'):401408/125, ('q24','q30'):802816/125, ('q25','q26'):100352/125, ('q25','q27'):200704/125, ('q25','q28'):401408/125, ('q25','q29'):802816/125, ('q25','q30'):1605632/125, ('q26','q27'):401408/125, ('q26','q28'):802816/125, ('q26','q29'):1605632/125, ('q26','q30'):3211264/125, ('q27','q28'):1605632/125, ('q27','q29'):3211264/125, ('q27','q30'):6422528/125, ('q28','q29'):6422528/125, ('q28','q30'):12845056/125, ('q29','q30'):25690112/125, ('q31','q32'):392/125, ('q31','q33'):784/125, ('q31','q34'):1568/125, ('q31','q35'):3136/125, ('q31','q36'):6272/125, ('q31','q37'):12544/125, ('q31','q38'):25088/125, ('q31','q39'):50176/125, ('q31','q40'):100352/125, ('q32','q33'):1568/125, ('q32','q34'):3136/125, ('q32','q35'):6272/125, ('q32','q36'):12544/125, ('q32','q37'):25088/125, ('q32','q38'):50176/125, ('q32','q39'):100352/125, ('q32','q40'):200704/125, ('q33','q34'):6272/125, ('q33','q35'):12544/125, ('q33','q36'):25088/125, ('q33','q37'):50176/125, ('q33','q38'):100352/125, ('q33','q39'):200704/125, ('q33','q40'):401408/125, ('q34','q35'):25088/125, ('q34','q36'):50176/125, ('q34','q37'):100352/125, ('q34','q38'):200704/125, ('q34','q39'):401408/125, ('q34','q40'):802816/125, ('q35','q36'):100352/125, ('q35','q37'):200704/125, ('q35','q38'):401408/125, ('q35','q39'):802816/125, ('q35','q40'):1605632/125, ('q36','q37'):401408/125, ('q36','q38'):802816/125, ('q36','q39'):1605632/125, ('q36','q40'):3211264/125, ('q37','q38'):1605632/125, ('q37','q39'):3211264/125, ('q37','q40'):6422528/125, ('q38','q39'):6422528/125, ('q38','q40'):12845056/125, ('q39','q40'):25690112/125}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9     energy num_oc. ...
10   0   0   0   0   0   1   1   0   0   0   0  1 ...  0   1192.704       1 ...
411  0   0   0   0   1   0   0   0   0   0   0  0 ...  0   2473.392       1 ...
0    0   0   0   1   0   1   1   0   0   0   0  1 ...  0   2617.392       1 ...
1    0   0   0   1   0   0   1   0   0   0   0  1 ...  0   3559.968       1 ...
20   1   0   0   0   1   0   0   1   0   0   0  1 ...  0   3627.376       1 ...
46   1   0   0   0   1   0   0   1   0   0   0  1 ...  0    3653.68       1 ...
2    1   0   0   0   1   0   0   1   0   0   0  1 ...  0   3870.512       1 ...
8    1   0   0   1   0   1   0   1   0   0   0  0 ...  0     3981.6       1 ...
3    0   0   1   1   0   0   1   0   0   0   0  1 ...  0   4168.368       1 ...
471  0   0   1   1   0   0   0   1   0   0   0  1 ...  0   4793.376       1 ...
25   1   0   0   1   1   0   0   0   0   0   0  0 ...  0    4860.08       1 ...
340  1   0   0   1   0   0   0   1   0  

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26, ('q2','q2'):72, ('q3','q3'):224, ('q4','q4'):768, ('q5','q5'):2816, ('q6','q6'):10752, ('q7','q7'):41984, ('q8','q8'):165888, ('q9','q9'):659456, ('q10','q10'):2629632, ('q11','q11'):-6, ('q12','q12'):8, ('q13','q13'):96, ('q14','q14'):512, ('q15','q15'):2304, ('q16','q16'):9728, ('q17','q17'):39936, ('q18','q18'):161792, ('q19','q19'):651264, ('q20','q20'):2613248, ('q21','q21'):-13, ('q22','q22'):-16, ('q23','q23'):8, ('q24','q24'):176, ('q25','q25'):992, ('q26','q26'):4544, ('q27','q27'):19328, ('q28','q28'):79616, ('q29','q29'):323072, ('q30','q30'):1301504, ('q31','q31'):23, ('q32','q32'):56, ('q33','q33'):152, ('q34','q34'):464, ('q35','q35'):1568, ('q36','q36'):5696, ('q37','q37'):21632, ('q38','q38'):84224, ('q39','q39'):332288, ('q40','q40'):1319936}

quadratic = {('q1','q2'):40, ('q1','q3'):80, ('q1','q4'):160, ('q1','q5'):320, ('q1','q6'):640, ('q1','q7'):1280, ('q1','q8'):2560, ('q1','q9'):5120, ('q1','q10'):10240, ('q1','q11'):-20, ('q1','q12'):-40, ('q1','q13'):-80, ('q1','q14'):-160, ('q1','q15'):-320, ('q1','q16'):-640, ('q1','q17'):-1280, ('q1','q18'):-2560, ('q1','q19'):-5120, ('q1','q20'):-10240, ('q1','q21'):2, ('q1','q22'):4, ('q1','q23'):8, ('q1','q24'):16, ('q1','q25'):32, ('q1','q26'):64, ('q1','q27'):128, ('q1','q28'):256, ('q1','q29'):512, ('q1','q30'):1024, ('q1','q31'):-2, ('q1','q32'):-4, ('q1','q33'):-8, ('q1','q34'):-16, ('q1','q35'):-32, ('q1','q36'):-64, ('q1','q37'):-128, ('q1','q38'):-256, ('q1','q39'):-512, ('q1','q40'):-1024, ('q2','q3'):160, ('q2','q4'):320, ('q2','q5'):640, ('q2','q6'):1280, ('q2','q7'):2560, ('q2','q8'):5120, ('q2','q9'):10240, ('q2','q10'):20480, ('q2','q11'):-40, ('q2','q12'):-80, ('q2','q13'):-160, ('q2','q14'):-320, ('q2','q15'):-640, ('q2','q16'):-1280, ('q2','q17'):-2560, ('q2','q18'):-5120, ('q2','q19'):-10240, ('q2','q20'):-20480, ('q2','q21'):4, ('q2','q22'):8, ('q2','q23'):16, ('q2','q24'):32, ('q2','q25'):64, ('q2','q26'):128, ('q2','q27'):256, ('q2','q28'):512, ('q2','q29'):1024, ('q2','q30'):2048, ('q2','q31'):-4, ('q2','q32'):-8, ('q2','q33'):-16, ('q2','q34'):-32, ('q2','q35'):-64, ('q2','q36'):-128, ('q2','q37'):-256, ('q2','q38'):-512, ('q2','q39'):-1024, ('q2','q40'):-2048, ('q3','q4'):640, ('q3','q5'):1280, ('q3','q6'):2560, ('q3','q7'):5120, ('q3','q8'):10240, ('q3','q9'):20480, ('q3','q10'):40960, ('q3','q11'):-80, ('q3','q12'):-160, ('q3','q13'):-320, ('q3','q14'):-640, ('q3','q15'):-1280, ('q3','q16'):-2560, ('q3','q17'):-5120, ('q3','q18'):-10240, ('q3','q19'):-20480, ('q3','q20'):-40960, ('q3','q21'):8, ('q3','q22'):16, ('q3','q23'):32, ('q3','q24'):64, ('q3','q25'):128, ('q3','q26'):256, ('q3','q27'):512, ('q3','q28'):1024, ('q3','q29'):2048, ('q3','q30'):4096, ('q3','q31'):-8, ('q3','q32'):-16, ('q3','q33'):-32, ('q3','q34'):-64, ('q3','q35'):-128, ('q3','q36'):-256, ('q3','q37'):-512, ('q3','q38'):-1024, ('q3','q39'):-2048, ('q3','q40'):-4096, ('q4','q5'):2560, ('q4','q6'):5120, ('q4','q7'):10240, ('q4','q8'):20480, ('q4','q9'):40960, ('q4','q10'):81920, ('q4','q11'):-160, ('q4','q12'):-320, ('q4','q13'):-640, ('q4','q14'):-1280, ('q4','q15'):-2560, ('q4','q16'):-5120, ('q4','q17'):-10240, ('q4','q18'):-20480, ('q4','q19'):-40960, ('q4','q20'):-81920, ('q4','q21'):16, ('q4','q22'):32, ('q4','q23'):64, ('q4','q24'):128, ('q4','q25'):256, ('q4','q26'):512, ('q4','q27'):1024, ('q4','q28'):2048, ('q4','q29'):4096, ('q4','q30'):8192, ('q4','q31'):-16, ('q4','q32'):-32, ('q4','q33'):-64, ('q4','q34'):-128, ('q4','q35'):-256, ('q4','q36'):-512, ('q4','q37'):-1024, ('q4','q38'):-2048, ('q4','q39'):-4096, ('q4','q40'):-8192, ('q5','q6'):10240, ('q5','q7'):20480, ('q5','q8'):40960, ('q5','q9'):81920, ('q5','q10'):163840, ('q5','q11'):-320, ('q5','q12'):-640, ('q5','q13'):-1280, ('q5','q14'):-2560, ('q5','q15'):-5120, ('q5','q16'):-10240, ('q5','q17'):-20480, ('q5','q18'):-40960, ('q5','q19'):-81920, ('q5','q20'):-163840, ('q5','q21'):32, ('q5','q22'):64, ('q5','q23'):128, ('q5','q24'):256, ('q5','q25'):512, ('q5','q26'):1024, ('q5','q27'):2048, ('q5','q28'):4096, ('q5','q29'):8192, ('q5','q30'):16384, ('q5','q31'):-32, ('q5','q32'):-64, ('q5','q33'):-128, ('q5','q34'):-256, ('q5','q35'):-512, ('q5','q36'):-1024, ('q5','q37'):-2048, ('q5','q38'):-4096, ('q5','q39'):-8192, ('q5','q40'):-16384, ('q6','q7'):40960, ('q6','q8'):81920, ('q6','q9'):163840, ('q6','q10'):327680, ('q6','q11'):-640, ('q6','q12'):-1280, ('q6','q13'):-2560, ('q6','q14'):-5120, ('q6','q15'):-10240, ('q6','q16'):-20480, ('q6','q17'):-40960, ('q6','q18'):-81920, ('q6','q19'):-163840, ('q6','q20'):-327680, ('q6','q21'):64, ('q6','q22'):128, ('q6','q23'):256, ('q6','q24'):512, ('q6','q25'):1024, ('q6','q26'):2048, ('q6','q27'):4096, ('q6','q28'):8192, ('q6','q29'):16384, ('q6','q30'):32768, ('q6','q31'):-64, ('q6','q32'):-128, ('q6','q33'):-256, ('q6','q34'):-512, ('q6','q35'):-1024, ('q6','q36'):-2048, ('q6','q37'):-4096, ('q6','q38'):-8192, ('q6','q39'):-16384, ('q6','q40'):-32768, ('q7','q8'):163840, ('q7','q9'):327680, ('q7','q10'):655360, ('q7','q11'):-1280, ('q7','q12'):-2560, ('q7','q13'):-5120, ('q7','q14'):-10240, ('q7','q15'):-20480, ('q7','q16'):-40960, ('q7','q17'):-81920, ('q7','q18'):-163840, ('q7','q19'):-327680, ('q7','q20'):-655360, ('q7','q21'):128, ('q7','q22'):256, ('q7','q23'):512, ('q7','q24'):1024, ('q7','q25'):2048, ('q7','q26'):4096, ('q7','q27'):8192, ('q7','q28'):16384, ('q7','q29'):32768, ('q7','q30'):65536, ('q7','q31'):-128, ('q7','q32'):-256, ('q7','q33'):-512, ('q7','q34'):-1024, ('q7','q35'):-2048, ('q7','q36'):-4096, ('q7','q37'):-8192, ('q7','q38'):-16384, ('q7','q39'):-32768, ('q7','q40'):-65536, ('q8','q9'):655360, ('q8','q10'):1310720, ('q8','q11'):-2560, ('q8','q12'):-5120, ('q8','q13'):-10240, ('q8','q14'):-20480, ('q8','q15'):-40960, ('q8','q16'):-81920, ('q8','q17'):-163840, ('q8','q18'):-327680, ('q8','q19'):-655360, ('q8','q20'):-1310720, ('q8','q21'):256, ('q8','q22'):512, ('q8','q23'):1024, ('q8','q24'):2048, ('q8','q25'):4096, ('q8','q26'):8192, ('q8','q27'):16384, ('q8','q28'):32768, ('q8','q29'):65536, ('q8','q30'):131072, ('q8','q31'):-256, ('q8','q32'):-512, ('q8','q33'):-1024, ('q8','q34'):-2048, ('q8','q35'):-4096, ('q8','q36'):-8192, ('q8','q37'):-16384, ('q8','q38'):-32768, ('q8','q39'):-65536, ('q8','q40'):-131072, ('q9','q10'):2621440, ('q9','q11'):-5120, ('q9','q12'):-10240, ('q9','q13'):-20480, ('q9','q14'):-40960, ('q9','q15'):-81920, ('q9','q16'):-163840, ('q9','q17'):-327680, ('q9','q18'):-655360, ('q9','q19'):-1310720, ('q9','q20'):-2621440, ('q9','q21'):512, ('q9','q22'):1024, ('q9','q23'):2048, ('q9','q24'):4096, ('q9','q25'):8192, ('q9','q26'):16384, ('q9','q27'):32768, ('q9','q28'):65536, ('q9','q29'):131072, ('q9','q30'):262144, ('q9','q31'):-512, ('q9','q32'):-1024, ('q9','q33'):-2048, ('q9','q34'):-4096, ('q9','q35'):-8192, ('q9','q36'):-16384, ('q9','q37'):-32768, ('q9','q38'):-65536, ('q9','q39'):-131072, ('q9','q40'):-262144, ('q10','q11'):-10240, ('q10','q12'):-20480, ('q10','q13'):-40960, ('q10','q14'):-81920, ('q10','q15'):-163840, ('q10','q16'):-327680, ('q10','q17'):-655360, ('q10','q18'):-1310720, ('q10','q19'):-2621440, ('q10','q20'):-5242880, ('q10','q21'):1024, ('q10','q22'):2048, ('q10','q23'):4096, ('q10','q24'):8192, ('q10','q25'):16384, ('q10','q26'):32768, ('q10','q27'):65536, ('q10','q28'):131072, ('q10','q29'):262144, ('q10','q30'):524288, ('q10','q31'):-1024, ('q10','q32'):-2048, ('q10','q33'):-4096, ('q10','q34'):-8192, ('q10','q35'):-16384, ('q10','q36'):-32768, ('q10','q37'):-65536, ('q10','q38'):-131072, ('q10','q39'):-262144, ('q10','q40'):-524288, ('q11','q12'):40, ('q11','q13'):80, ('q11','q14'):160, ('q11','q15'):320, ('q11','q16'):640, ('q11','q17'):1280, ('q11','q18'):2560, ('q11','q19'):5120, ('q11','q20'):10240, ('q11','q21'):-2, ('q11','q22'):-4, ('q11','q23'):-8, ('q11','q24'):-16, ('q11','q25'):-32, ('q11','q26'):-64, ('q11','q27'):-128, ('q11','q28'):-256, ('q11','q29'):-512, ('q11','q30'):-1024, ('q11','q31'):2, ('q11','q32'):4, ('q11','q33'):8, ('q11','q34'):16, ('q11','q35'):32, ('q11','q36'):64, ('q11','q37'):128, ('q11','q38'):256, ('q11','q39'):512, ('q11','q40'):1024, ('q12','q13'):160, ('q12','q14'):320, ('q12','q15'):640, ('q12','q16'):1280, ('q12','q17'):2560, ('q12','q18'):5120, ('q12','q19'):10240, ('q12','q20'):20480, ('q12','q21'):-4, ('q12','q22'):-8, ('q12','q23'):-16, ('q12','q24'):-32, ('q12','q25'):-64, ('q12','q26'):-128, ('q12','q27'):-256, ('q12','q28'):-512, ('q12','q29'):-1024, ('q12','q30'):-2048, ('q12','q31'):4, ('q12','q32'):8, ('q12','q33'):16, ('q12','q34'):32, ('q12','q35'):64, ('q12','q36'):128, ('q12','q37'):256, ('q12','q38'):512, ('q12','q39'):1024, ('q12','q40'):2048, ('q13','q14'):640, ('q13','q15'):1280, ('q13','q16'):2560, ('q13','q17'):5120, ('q13','q18'):10240, ('q13','q19'):20480, ('q13','q20'):40960, ('q13','q21'):-8, ('q13','q22'):-16, ('q13','q23'):-32, ('q13','q24'):-64, ('q13','q25'):-128, ('q13','q26'):-256, ('q13','q27'):-512, ('q13','q28'):-1024, ('q13','q29'):-2048, ('q13','q30'):-4096, ('q13','q31'):8, ('q13','q32'):16, ('q13','q33'):32, ('q13','q34'):64, ('q13','q35'):128, ('q13','q36'):256, ('q13','q37'):512, ('q13','q38'):1024, ('q13','q39'):2048, ('q13','q40'):4096, ('q14','q15'):2560, ('q14','q16'):5120, ('q14','q17'):10240, ('q14','q18'):20480, ('q14','q19'):40960, ('q14','q20'):81920, ('q14','q21'):-16, ('q14','q22'):-32, ('q14','q23'):-64, ('q14','q24'):-128, ('q14','q25'):-256, ('q14','q26'):-512, ('q14','q27'):-1024, ('q14','q28'):-2048, ('q14','q29'):-4096, ('q14','q30'):-8192, ('q14','q31'):16, ('q14','q32'):32, ('q14','q33'):64, ('q14','q34'):128, ('q14','q35'):256, ('q14','q36'):512, ('q14','q37'):1024, ('q14','q38'):2048, ('q14','q39'):4096, ('q14','q40'):8192, ('q15','q16'):10240, ('q15','q17'):20480, ('q15','q18'):40960, ('q15','q19'):81920, ('q15','q20'):163840, ('q15','q21'):-32, ('q15','q22'):-64, ('q15','q23'):-128, ('q15','q24'):-256, ('q15','q25'):-512, ('q15','q26'):-1024, ('q15','q27'):-2048, ('q15','q28'):-4096, ('q15','q29'):-8192, ('q15','q30'):-16384, ('q15','q31'):32, ('q15','q32'):64, ('q15','q33'):128, ('q15','q34'):256, ('q15','q35'):512, ('q15','q36'):1024, ('q15','q37'):2048, ('q15','q38'):4096, ('q15','q39'):8192, ('q15','q40'):16384, ('q16','q17'):40960, ('q16','q18'):81920, ('q16','q19'):163840, ('q16','q20'):327680, ('q16','q21'):-64, ('q16','q22'):-128, ('q16','q23'):-256, ('q16','q24'):-512, ('q16','q25'):-1024, ('q16','q26'):-2048, ('q16','q27'):-4096, ('q16','q28'):-8192, ('q16','q29'):-16384, ('q16','q30'):-32768, ('q16','q31'):64, ('q16','q32'):128, ('q16','q33'):256, ('q16','q34'):512, ('q16','q35'):1024, ('q16','q36'):2048, ('q16','q37'):4096, ('q16','q38'):8192, ('q16','q39'):16384, ('q16','q40'):32768, ('q17','q18'):163840, ('q17','q19'):327680, ('q17','q20'):655360, ('q17','q21'):-128, ('q17','q22'):-256, ('q17','q23'):-512, ('q17','q24'):-1024, ('q17','q25'):-2048, ('q17','q26'):-4096, ('q17','q27'):-8192, ('q17','q28'):-16384, ('q17','q29'):-32768, ('q17','q30'):-65536, ('q17','q31'):128, ('q17','q32'):256, ('q17','q33'):512, ('q17','q34'):1024, ('q17','q35'):2048, ('q17','q36'):4096, ('q17','q37'):8192, ('q17','q38'):16384, ('q17','q39'):32768, ('q17','q40'):65536, ('q18','q19'):655360, ('q18','q20'):1310720, ('q18','q21'):-256, ('q18','q22'):-512, ('q18','q23'):-1024, ('q18','q24'):-2048, ('q18','q25'):-4096, ('q18','q26'):-8192, ('q18','q27'):-16384, ('q18','q28'):-32768, ('q18','q29'):-65536, ('q18','q30'):-131072, ('q18','q31'):256, ('q18','q32'):512, ('q18','q33'):1024, ('q18','q34'):2048, ('q18','q35'):4096, ('q18','q36'):8192, ('q18','q37'):16384, ('q18','q38'):32768, ('q18','q39'):65536, ('q18','q40'):131072, ('q19','q20'):2621440, ('q19','q21'):-512, ('q19','q22'):-1024, ('q19','q23'):-2048, ('q19','q24'):-4096, ('q19','q25'):-8192, ('q19','q26'):-16384, ('q19','q27'):-32768, ('q19','q28'):-65536, ('q19','q29'):-131072, ('q19','q30'):-262144, ('q19','q31'):512, ('q19','q32'):1024, ('q19','q33'):2048, ('q19','q34'):4096, ('q19','q35'):8192, ('q19','q36'):16384, ('q19','q37'):32768, ('q19','q38'):65536, ('q19','q39'):131072, ('q19','q40'):262144, ('q20','q21'):-1024, ('q20','q22'):-2048, ('q20','q23'):-4096, ('q20','q24'):-8192, ('q20','q25'):-16384, ('q20','q26'):-32768, ('q20','q27'):-65536, ('q20','q28'):-131072, ('q20','q29'):-262144, ('q20','q30'):-524288, ('q20','q31'):1024, ('q20','q32'):2048, ('q20','q33'):4096, ('q20','q34'):8192, ('q20','q35'):16384, ('q20','q36'):32768, ('q20','q37'):65536, ('q20','q38'):131072, ('q20','q39'):262144, ('q20','q40'):524288, ('q21','q22'):20, ('q21','q23'):40, ('q21','q24'):80, ('q21','q25'):160, ('q21','q26'):320, ('q21','q27'):640, ('q21','q28'):1280, ('q21','q29'):2560, ('q21','q30'):5120, ('q21','q31'):-10, ('q21','q32'):-20, ('q21','q33'):-40, ('q21','q34'):-80, ('q21','q35'):-160, ('q21','q36'):-320, ('q21','q37'):-640, ('q21','q38'):-1280, ('q21','q39'):-2560, ('q21','q40'):-5120, ('q22','q23'):80, ('q22','q24'):160, ('q22','q25'):320, ('q22','q26'):640, ('q22','q27'):1280, ('q22','q28'):2560, ('q22','q29'):5120, ('q22','q30'):10240, ('q22','q31'):-20, ('q22','q32'):-40, ('q22','q33'):-80, ('q22','q34'):-160, ('q22','q35'):-320, ('q22','q36'):-640, ('q22','q37'):-1280, ('q22','q38'):-2560, ('q22','q39'):-5120, ('q22','q40'):-10240, ('q23','q24'):320, ('q23','q25'):640, ('q23','q26'):1280, ('q23','q27'):2560, ('q23','q28'):5120, ('q23','q29'):10240, ('q23','q30'):20480, ('q23','q31'):-40, ('q23','q32'):-80, ('q23','q33'):-160, ('q23','q34'):-320, ('q23','q35'):-640, ('q23','q36'):-1280, ('q23','q37'):-2560, ('q23','q38'):-5120, ('q23','q39'):-10240, ('q23','q40'):-20480, ('q24','q25'):1280, ('q24','q26'):2560, ('q24','q27'):5120, ('q24','q28'):10240, ('q24','q29'):20480, ('q24','q30'):40960, ('q24','q31'):-80, ('q24','q32'):-160, ('q24','q33'):-320, ('q24','q34'):-640, ('q24','q35'):-1280, ('q24','q36'):-2560, ('q24','q37'):-5120, ('q24','q38'):-10240, ('q24','q39'):-20480, ('q24','q40'):-40960, ('q25','q26'):5120, ('q25','q27'):10240, ('q25','q28'):20480, ('q25','q29'):40960, ('q25','q30'):81920, ('q25','q31'):-160, ('q25','q32'):-320, ('q25','q33'):-640, ('q25','q34'):-1280, ('q25','q35'):-2560, ('q25','q36'):-5120, ('q25','q37'):-10240, ('q25','q38'):-20480, ('q25','q39'):-40960, ('q25','q40'):-81920, ('q26','q27'):20480, ('q26','q28'):40960, ('q26','q29'):81920, ('q26','q30'):163840, ('q26','q31'):-320, ('q26','q32'):-640, ('q26','q33'):-1280, ('q26','q34'):-2560, ('q26','q35'):-5120, ('q26','q36'):-10240, ('q26','q37'):-20480, ('q26','q38'):-40960, ('q26','q39'):-81920, ('q26','q40'):-163840, ('q27','q28'):81920, ('q27','q29'):163840, ('q27','q30'):327680, ('q27','q31'):-640, ('q27','q32'):-1280, ('q27','q33'):-2560, ('q27','q34'):-5120, ('q27','q35'):-10240, ('q27','q36'):-20480, ('q27','q37'):-40960, ('q27','q38'):-81920, ('q27','q39'):-163840, ('q27','q40'):-327680, ('q28','q29'):327680, ('q28','q30'):655360, ('q28','q31'):-1280, ('q28','q32'):-2560, ('q28','q33'):-5120, ('q28','q34'):-10240, ('q28','q35'):-20480, ('q28','q36'):-40960, ('q28','q37'):-81920, ('q28','q38'):-163840, ('q28','q39'):-327680, ('q28','q40'):-655360, ('q29','q30'):1310720, ('q29','q31'):-2560, ('q29','q32'):-5120, ('q29','q33'):-10240, ('q29','q34'):-20480, ('q29','q35'):-40960, ('q29','q36'):-81920, ('q29','q37'):-163840, ('q29','q38'):-327680, ('q29','q39'):-655360, ('q29','q40'):-1310720, ('q30','q31'):-5120, ('q30','q32'):-10240, ('q30','q33'):-20480, ('q30','q34'):-40960, ('q30','q35'):-81920, ('q30','q36'):-163840, ('q30','q37'):-327680, ('q30','q38'):-655360, ('q30','q39'):-1310720, ('q30','q40'):-2621440, ('q31','q32'):20, ('q31','q33'):40, ('q31','q34'):80, ('q31','q35'):160, ('q31','q36'):320, ('q31','q37'):640, ('q31','q38'):1280, ('q31','q39'):2560, ('q31','q40'):5120, ('q32','q33'):80, ('q32','q34'):160, ('q32','q35'):320, ('q32','q36'):640, ('q32','q37'):1280, ('q32','q38'):2560, ('q32','q39'):5120, ('q32','q40'):10240, ('q33','q34'):320, ('q33','q35'):640, ('q33','q36'):1280, ('q33','q37'):2560, ('q33','q38'):5120, ('q33','q39'):10240, ('q33','q40'):20480, ('q34','q35'):1280, ('q34','q36'):2560, ('q34','q37'):5120, ('q34','q38'):10240, ('q34','q39'):20480, ('q34','q40'):40960, ('q35','q36'):5120, ('q35','q37'):10240, ('q35','q38'):20480, ('q35','q39'):40960, ('q35','q40'):81920, ('q36','q37'):20480, ('q36','q38'):40960, ('q36','q39'):81920, ('q36','q40'):163840, ('q37','q38'):81920, ('q37','q39'):163840, ('q37','q40'):327680, ('q38','q39'):327680, ('q38','q40'):655360, ('q39','q40'):1310720}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q13 q14 q15 q16 q17 q18 q19 q2 ... q9    energy num_oc. ...
437  0   0   0   0   0   0   1   1   0   0   1  0 ...  1    3871.0       1 ...
745  1   0   0   1   0   0   0   0   0   1   1  1 ...  1    4767.0       1 ...
932  1   0   0   1   1   1   0   1   1   1   0  0 ...  0    4923.0       1 ...
522  0   1   0   0   1   0   1   1   1   1   0  0 ...  0    6039.0       1 ...
859  1   0   0   0   0   0   0   0   0   1   1  1 ...  1    6363.0       1 ...
682  1   0   0   1   0   0   1   0   0   1   1  0 ...  1    9674.0       1 ...
920  1   0   0   0   0   1   1   0   0   1   1  0 ...  1   12110.0       1 ...
838  1   1   0   0   1   0   1   1   1   0   0  1 ...  0   14922.0       1 ...
322  1   0   1   1   1   0   0   1   0   1   1  1 ...  1   19027.0       1 ...
50   1   1   0   0   0   0   0   1   1   0   0  0 ...  0   19055.0       1 ...
115  1   1   0   0   0   0   1   1   1   0   0  1 ...  0   19299.0       1 ...
849  1   1   0   0   0   1   1   1   1   1   0  1 ..